**Table of contents**<a id='toc0_'></a>    
- [Data Prepare](#toc1_1_)    
  - [Clean Data](#toc1_2_)    
  - [Modeling](#toc1_3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[Data Prepare](#toc0_)

In [84]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [85]:
raw_file = r"C:\CODES\Formacao_Dados_DNC\data\raw_data\ICPSR_38050\DS0001\38050-0001-Data.tsv"

df = pd.read_csv(raw_file, sep="\t")
df.head()

C:\Users\Meu Computador\AppData\Local\Temp\ipykernel_17568\2999172331.py:3: DtypeWarning: Columns (9,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(raw_file, sep="\t")


,CASEID,NAME,PID,CATEGORY,SUBCATEGORY,PROJECT_PAGE_LOCATION_NAME,PROJECT_PAGE_LOCATION_STATE,PROJECT_PAGE_LOCATION_COUNTY,PROJECT_PAGE_LOCATION_COUNTRY,UID,PROJECT_CURRENCY,GOAL_IN_ORIGINAL_CURRENCY,PLEDGED_IN_ORIGINAL_CURRENCY,GOAL_IN_USD,PLEDGED_IN_USD,BACKERS_COUNT,LAUNCHED_DATE,DEADLINE_DATE,STATE,URL_NAME
0,1,MASKED BY ICPSR,2137925650,11,301,London,England,Greater London,,1076478145,USD,"100,000,000",0,"$100,000,000",$0,0,8/11/2016,10/10/2016,canceled,MASKED BY ICPSR
1,2,MASKED BY ICPSR,1501531085,11,296,Los Angeles,CA,Los Angeles,,224946798,USD,"100,000,000",85,"$100,000,000",$85,4,12/19/2019,2/14/2020,canceled,MASKED BY ICPSR
2,3,MASKED BY ICPSR,953415668,16,51,Mexico,Baja California,Tijuana,,1772203542,MXN,"100,000,000",10,"$5,219,374",$1,1,3/1/2017,3/22/2017,failed,MASKED BY ICPSR
3,4,MASKED BY ICPSR,1371386304,18,18,Columbus,OH,Franklin,,1373465389,USD,"100,000,000",1,"$100,000,000",$1,1,6/4/2018,7/5/2018,canceled,MASKED BY ICPSR
4,5,MASKED BY ICPSR,1720842777,1,22,Toronto,ON,Toronto,,1455666383,CAD,"100,000,000",0,"$80,610,122",$0,0,5/1/2015,6/30/2015,failed,MASKED BY ICPSR


In [86]:
df.columns

Index(['CASEID', 'NAME', 'PID', 'CATEGORY', 'SUBCATEGORY',
       'PROJECT_PAGE_LOCATION_NAME', 'PROJECT_PAGE_LOCATION_STATE',
       'PROJECT_PAGE_LOCATION_COUNTY', 'PROJECT_PAGE_LOCATION_COUNTRY', 'UID',
       'PROJECT_CURRENCY', 'GOAL_IN_ORIGINAL_CURRENCY',
       'PLEDGED_IN_ORIGINAL_CURRENCY', 'GOAL_IN_USD', 'PLEDGED_IN_USD',
       'BACKERS_COUNT', 'LAUNCHED_DATE', 'DEADLINE_DATE', 'STATE', 'URL_NAME'],
      dtype='object')

In [87]:
df = df.drop(columns=['CASEID',
                      'NAME',
                      'URL_NAME',
                      'PROJECT_CURRENCY',
                      'PROJECT_PAGE_LOCATION_COUNTRY', 
                      'PROJECT_PAGE_LOCATION_STATE',
                      'PROJECT_PAGE_LOCATION_COUNTY',
                      'PROJECT_PAGE_LOCATION_NAME', #reduzindo para melhorar performance
                      'UID',
                      'GOAL_IN_ORIGINAL_CURRENCY',
                      'PLEDGED_IN_ORIGINAL_CURRENCY',
                      ])

In [88]:
df.head()

,PID,CATEGORY,SUBCATEGORY,GOAL_IN_USD,PLEDGED_IN_USD,BACKERS_COUNT,LAUNCHED_DATE,DEADLINE_DATE,STATE
0,2137925650,11,301,"$100,000,000",$0,0,8/11/2016,10/10/2016,canceled
1,1501531085,11,296,"$100,000,000",$85,4,12/19/2019,2/14/2020,canceled
2,953415668,16,51,"$5,219,374",$1,1,3/1/2017,3/22/2017,failed
3,1371386304,18,18,"$100,000,000",$1,1,6/4/2018,7/5/2018,canceled
4,1720842777,1,22,"$80,610,122",$0,0,5/1/2015,6/30/2015,failed


In [89]:
df.dtypes

PID                int64
CATEGORY           int64
SUBCATEGORY        int64
GOAL_IN_USD       object
PLEDGED_IN_USD    object
BACKERS_COUNT     object
LAUNCHED_DATE     object
DEADLINE_DATE     object
STATE             object
dtype: object

## <a id='toc1_2_'></a>[Clean Data](#toc0_)

In [90]:
df.columns

Index(['PID', 'CATEGORY', 'SUBCATEGORY', 'GOAL_IN_USD', 'PLEDGED_IN_USD',
       'BACKERS_COUNT', 'LAUNCHED_DATE', 'DEADLINE_DATE', 'STATE'],
      dtype='object')

In [91]:
# Trata coluna de valores
df['GOAL_IN_USD'] = df['GOAL_IN_USD'].apply(lambda x: x.strip('$'))
df['GOAL_IN_USD'] = df['GOAL_IN_USD'].apply(lambda x: x.replace(',', '')).astype(float)

df['PLEDGED_IN_USD'] = df['PLEDGED_IN_USD'].apply(lambda x: x.strip('$'))
df['PLEDGED_IN_USD'] = df['PLEDGED_IN_USD'].apply(lambda x: x.replace(',', '')).astype(float)

In [92]:
#definindo colunas de data
df['LAUNCHED_DATE'] = pd.to_datetime(df['LAUNCHED_DATE'], format='mixed')
df['DEADLINE_DATE'] = pd.to_datetime(df['DEADLINE_DATE'], format='mixed')

In [93]:
df.dtypes

PID                        int64
CATEGORY                   int64
SUBCATEGORY                int64
GOAL_IN_USD              float64
PLEDGED_IN_USD           float64
BACKERS_COUNT             object
LAUNCHED_DATE     datetime64[ns]
DEADLINE_DATE     datetime64[ns]
STATE                     object
dtype: object

In [94]:
df['time_range_project'] = (df['DEADLINE_DATE'] - df['LAUNCHED_DATE']).dt.days

In [95]:
df.head()

,PID,CATEGORY,SUBCATEGORY,GOAL_IN_USD,PLEDGED_IN_USD,BACKERS_COUNT,LAUNCHED_DATE,DEADLINE_DATE,STATE,time_range_project
0,2137925650,11,301,100000000.0,0.0,0,2016-08-11,2016-10-10,canceled,60
1,1501531085,11,296,100000000.0,85.0,4,2019-12-19,2020-02-14,canceled,57
2,953415668,16,51,5219374.0,1.0,1,2017-03-01,2017-03-22,failed,21
3,1371386304,18,18,100000000.0,1.0,1,2018-06-04,2018-07-05,canceled,31
4,1720842777,1,22,80610122.0,0.0,0,2015-05-01,2015-06-30,failed,60


In [96]:
raw_file2 = r"C:\CODES\Formacao_Dados_DNC\data\raw_data\ICPSR_38050\DS0003\38050-0003-Data.tsv"
df2 = pd.read_csv(raw_file2, sep="\t")
df2.head()

C:\Users\Meu Computador\AppData\Local\Temp\ipykernel_17568\268374242.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv(raw_file2, sep="\t")


,CASEID,PID,USER_LOCATION_STATE,USER_LOCATION_COUNTRY,COUNT,TOTAL_AMOUNT_IN_USD
0,1,1955357092,CA,US,"25,058","$914,569"
1,2,1782528227,,,"18,509","$272,378"
2,3,1386523707,CA,US,"17,698","$641,466"
3,4,557230947,CA,US,"16,860","$949,476"
4,5,1755266685,CA,US,"12,140","$821,456"


In [97]:
df2.columns

Index(['CASEID', 'PID', 'USER_LOCATION_STATE', 'USER_LOCATION_COUNTRY',
       'COUNT', 'TOTAL_AMOUNT_IN_USD'],
      dtype='object')

In [98]:
df2 = df2.drop(columns=['CASEID', 'USER_LOCATION_STATE', 'TOTAL_AMOUNT_IN_USD'])

In [99]:
#df2.rename(columns={'PID': 'PROJECT_ID'}, inplace=True)
df2.head()

,PID,USER_LOCATION_COUNTRY,COUNT
0,1955357092,US,"25,058"
1,1782528227,,"18,509"
2,1386523707,US,"17,698"
3,557230947,US,"16,860"
4,1755266685,US,"12,140"


In [100]:
df = df.merge(df2, how='right', on=['PID'])

In [101]:
df.head()

,PID,CATEGORY,SUBCATEGORY,GOAL_IN_USD,PLEDGED_IN_USD,BACKERS_COUNT,LAUNCHED_DATE,DEADLINE_DATE,STATE,time_range_project,USER_LOCATION_COUNTRY,COUNT
0,1955357092,12,34,10000.0,8782572.0,219382,2015-01-20,2015-02-20,successful,31,US,"25,058"
1,1782528227,11,30,172487.0,567814.0,31536,2019-10-09,2019-11-07,successful,29,,"18,509"
2,1386523707,7,28,15000.0,6465690.0,154926,2016-08-30,2016-10-20,successful,51,US,"17,698"
3,557230947,16,342,1000000.0,5408917.0,105857,2014-05-28,2014-07-02,successful,35,US,"16,860"
4,1755266685,11,31,2000000.0,5702153.0,91585,2013-03-13,2013-04-13,successful,31,US,"12,140"


In [102]:
df.dtypes

PID                               int64
CATEGORY                          int64
SUBCATEGORY                       int64
GOAL_IN_USD                     float64
PLEDGED_IN_USD                  float64
BACKERS_COUNT                    object
LAUNCHED_DATE            datetime64[ns]
DEADLINE_DATE            datetime64[ns]
STATE                            object
time_range_project                int64
USER_LOCATION_COUNTRY            object
COUNT                            object
dtype: object

In [103]:
df['BACKERS_COUNT'] = df['BACKERS_COUNT'].str.strip().replace('', np.nan).fillna(0).astype(int)

In [104]:
df['COUNT'] = df['COUNT'].str.replace(',', '').fillna(0).astype(int)

In [105]:
# Convert categorical columns to category data type
#categorical_cols = ['PROJECT_PAGE_LOCATION_NAME']
#df[categorical_cols] = df[categorical_cols].astype('category')

# Perform one-hot encoding
#df = pd.get_dummies(df, columns=categorical_cols, sparse=True)


In [106]:
# Convert categorical columns to category data type
categorical_cols = ['USER_LOCATION_COUNTRY']
df[categorical_cols] = df[categorical_cols].astype('category')

# Perform one-hot encoding
df = pd.get_dummies(df, columns=categorical_cols, sparse=True)


In [107]:
df.head()

,PID,CATEGORY,SUBCATEGORY,GOAL_IN_USD,PLEDGED_IN_USD,BACKERS_COUNT,LAUNCHED_DATE,DEADLINE_DATE,STATE,time_range_project,...,USER_LOCATION_COUNTRY_VN,USER_LOCATION_COUNTRY_VU,USER_LOCATION_COUNTRY_WF,USER_LOCATION_COUNTRY_WS,USER_LOCATION_COUNTRY_XK,USER_LOCATION_COUNTRY_YE,USER_LOCATION_COUNTRY_YT,USER_LOCATION_COUNTRY_ZA,USER_LOCATION_COUNTRY_ZM,USER_LOCATION_COUNTRY_ZW
0,1955357092,12,34,10000.0,8782572.0,0,2015-01-20,2015-02-20,successful,31,...,False,False,False,False,False,False,False,False,False,False
1,1782528227,11,30,172487.0,567814.0,0,2019-10-09,2019-11-07,successful,29,...,False,False,False,False,False,False,False,False,False,False
2,1386523707,7,28,15000.0,6465690.0,0,2016-08-30,2016-10-20,successful,51,...,False,False,False,False,False,False,False,False,False,False
3,557230947,16,342,1000000.0,5408917.0,0,2014-05-28,2014-07-02,successful,35,...,False,False,False,False,False,False,False,False,False,False
4,1755266685,11,31,2000000.0,5702153.0,0,2013-03-13,2013-04-13,successful,31,...,False,False,False,False,False,False,False,False,False,False


## <a id='toc1_3_'></a>[Modeling](#toc0_)

In [109]:
y = df['STATE']
x = df.drop(columns=['STATE', 'PID', 'LAUNCHED_DATE', 'DEADLINE_DATE'])

In [111]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [112]:
classification = DecisionTreeClassifier(random_state=42)

In [113]:
classification_model = classification.fit(X_train, y_train)

c:\CODES\Formacao_Dados_DNC\.venv\Lib\site-packages\sklearn\utils\validation.py:869: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


In [114]:
classification_model.score(X_test, y_test)

c:\CODES\Formacao_Dados_DNC\.venv\Lib\site-packages\sklearn\utils\validation.py:869: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


0.9972063748954387

## Evaluation

- A métrica parece ser interessante, mas utilizamos valores de arrecadação, valor este que não teríamos em um cenário real.
- Fazer uma padronização para 0/1 na variável alvo
- Remover valores finais arrecadados das features do modelo


df['STATE'].unique()

In [ ]:
df['STATE'] = df['STATE'].apply(lambda x: 1 if x == 'successful' else 0)